# CNN model

## Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

## Fetching data

In [ ]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Constants

In [ ]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
NUMOF_CLASSES = 10

### model dependent constants
the following belong to model 1

In [ ]:
TRAIN_DATASET_SIZE = X_train.shape[0]
VALIDATION_DATASET_SIZE = int(0.15 * TRAIN_DATASET_SIZE)
TEST_DATASET_SIZE = X_test.shape[0]
BATCH_SIZE = 64
BUFFER_SIZE = int(1.1*TRAIN_DATASET_SIZE) # the buffer size for shuffling the dataset.
# the size should be greater than or equal to the dataset in order to have perfect shuffling.

## Dataset examples

In [ ]:
X_train
plt.figure(figsize=(27, 7))
for i in range(60):
  ax = plt.subplot(4, 15, i + 1)
  plt.imshow(X_train[i],cmap='gray')
  plt.title(Y_train[i])
  plt.axis("off")

In [ ]:
new_Y = keras.utils.to_categorical(Y_train, NUMOF_CLASSES)
for i in range(18):
  print(new_Y[i])
  print(Y_train[i])

## Dataset preprocessing

In [ ]:
#Y_train = keras.utils.to_categorical(Y_train, NUMOF_CLASSES)
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_ds = dataset.skip(VALIDATION_DATASET_SIZE)
valid_ds = dataset.take(VALIDATION_DATASET_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
train_ds

## Model 1

In [ ]:
# locally connected convolutions feature extractors.
modelC = keras.models.Sequential()
modelC.add(keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 1))) # grayscale, change to 3 for RGB.
modelC.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
modelC.add(keras.layers.MaxPooling2D((2, 2)))
modelC.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
modelC.add(keras.layers.MaxPooling2D((2, 2)))
modelC.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

# fully connected layers
modelC.add(keras.layers.Flatten()) # 3D tensor to 1D vector.
modelC.add(keras.layers.Dense(NUMOF_CLASSES))
modelC.add(keras.layers.Dense(128, activation='relu')) # hidden layer with fully connect neurons.
modelC.add(keras.layers.Dense(32, activation='relu')) # hidden layer with fully connect neurons. adding the bottleneck layer to reduce the number of parameters.
modelC.add(keras.layers.Dense(128, activation='relu')) # hidden layer with fully connect neurons.
modelC.add(keras.layers.Dense(NUMOF_CLASSES))
# output layer which indicates the number of classes.

modelC.summary()

In [ ]:
modelC.compile(optimizer='adam',
              #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              loss="categorical_crossentropy",
              metrics=['accuracy'],
              run_eagerly=True)

In [ ]:
epochs=15

historyC = modelC.fit(
  train_ds,
  validation_data=valid_ds,
  epochs=epochs
)

In [ ]:
plt.figure()
plt.plot(historyC.history['accuracy'], label='accuracy')
plt.plot(historyC.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
acc_C = historyC.history['accuracy']
val_acc_C = historyC.history['val_accuracy']

loss_C = historyC.history['loss']
val_loss_C = historyC.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc_C, label='Training Accuracy')
plt.plot(epochs_range, val_acc_C, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss_C, label='Training Loss')
plt.plot(epochs_range, val_loss_C, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Model 2

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
model = keras.Sequential([
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
      ])

model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", score[0])
print("Test accuracy:", score[1])